# Final Project

## Data Exploration

In [2]:
from xml.dom.minidom import parse
from math import log
import xml
import os
import sys
import csv 
import json
import re
import requests
import math
import xml.dom.minidom
data_path = r"C:\Programming\Python\Programs_NLP\wikihowcom-20141208-current.xml"
if sys.version_info[0] < 3:
    raise Exception("Python 3 or a more recent version is required.")

batches = []
batch_size = 100
batch_count = 0
lines = 12328007

In [18]:
def process_batch(batches):

        with open('temp.xml', 'w') as f:
            f.write('<mediawiki xmlns="http://www.mediawiki.org/xml/export-0.8/" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.mediawiki.org/xml/export-0.8/ http://www.mediawiki.org/xml/export-0.8.xsd" version="0.8" xml:lang="en">\n')
            for item in batches:
                f.write("%s" % item.encode('utf8'))
            f.close()

        with open('temp.xml', 'a') as f:
            f.write('</mediawiki>')
            f.close()
        data_dict = parse_data('test.xml')
        keys = [key for key in data_dict]

        posts = []
        for key in keys:
            posts.append(parse_(key, data_dict))

        with open('temp.json', 'w') as outfile:
            for pa in posts:
                json.dump({ "index" : {"_index": "wikihow", "_type": "page", "_id": pa["id"]}}, outfile)
                outfile.write('\n')
                json.dump(pa, outfile)
                outfile.write('\n')
            outfile.close()
        bulk_data = []

        for pa in posts:
            bulk_data.append({"index" : {"_index": "wikihow", "_type": "page", "_id": pa["id"]}})
            bulk_data.append(json.dumps(pa, ensure_ascii=False))

        print("Batch Number: " + str(batch_count) + " Processed")
        print("Indexing Batch Number: " + str(batch_count) + "...")
# 		config = {'host': 'localhost'}
# 		es = elasticsearch.Elasticsearch([config,], timeout=300)
# 		res = es.bulk(index = 'wikihow', body = bulk_data)
        #print(res)
        # check data is in there, and structure in there
        #print("Test Import...")
        #print(json.dumps(es.search(body={"query": {"match_all": {}}}, index = 'wikihow')))

def parse_data(file):
    file_dict = {}
    # Open XML document using minidom parser
    DOMTree = xml.dom.minidom.parse(file)
    mediawiki = DOMTree.documentElement
    #if mediawiki.hasAttribute("xmlns"):
        #print("Root element : %s" % mediawiki.getAttribute("xmlns"))
    # Get all the pages on file
    pages = mediawiki.getElementsByTagName("page")
    # Get high level namespace information.
    for page in pages:
        page_dom = page.getElementsByTagName('text')[0]
        dom_text = page_dom.childNodes[0].data
        if str(dom_text[0]) == '#':
            pass
        else:
            id_ = page.getElementsByTagName('id')[0]
            id_ = id_.childNodes[0].data
            title = page.getElementsByTagName('title')[0]
            title = title.childNodes[0].data
            text = page.getElementsByTagName('text')[0]
            text = text.childNodes[0].data
            file_dict[int(id_)] = [title, text]
    return file_dict


#data_dict = parse_data(data_path)

def structure(page_id, file_dict):
    file_ = file_dict[page_id][1]
    lines = file_.split("\n")
    lines_ = []
    for line in lines:
        if line != "":
            lines_.append(line)
    intro = lines_[0]
    try:
        if "Category" in lines_[1]:
            cat = lines_[1]
            if ":" in cat:
                cat = cat.split(":")[1][:-2]
        else:
            cat = None
    except IndexError:
        cat = None
    try:
        if "Category" in lines[2] and cat is not None:
            cat = cat + "," + re.sub('[^A-Za-z0-9\s\.|\:\*]+', '', lines[2]).split(":")[1]
            process = lines_[3:]
        else:
            process = lines_[2:]
    except IndexError:
        process = None

    return intro, cat, process

def merge(data_list):
    try:
        data_list = " ".join(data_list)
        data_list = data_list.split(" ==")
        data_list = list(zip(*[iter(data_list)] * 2))
    except TypeError:
        data_list = []
    return data_list

def construct_dict(data_list):
    textdict = {}
    steps_dict = {}
    #construct step dictt
    try:
        if data_list[0][1] != "":
            title = re.sub(r'([^\:\w])+', '', data_list[0][0]) 
            #print(title)
            steps = data_list[0][1].split(" #")
            steps = [st for st in steps if st != ""]
            step_index = ["step" + str(number + 1) for number in range(len(steps))]
            step_data = [ste.split("<br><br>") for ste in steps]
            step_new =  []
            for step in step_data:
                step_new.append(step[0])  
            step_data = step_new
            steps_dict = dict(zip(step_index, step_data))
            #return steps_dict
            textdict["steps"] = steps_dict
        subsections = []
        for sect in data_list[1:]:
            if sect == "":
                sect.pop(0)
                #print(sect)
            title = sect[0]
            title = re.sub(r'([^\:\w_])+', '', title)
            title  = "".join([t for t in title if t != " "])
            #print(sect[1:])
            content = sect[1]
            if content[1] == "*":
                subs = content.split("*")
                subs = [s for s in subs if s != " "]
                subs = [re.sub('[^A-Za-z0-9\s\.|\:\*]+', '', string) for string in subs]
                textdict[title] = subs
        return textdict
    except IndexError:
        return {}


def parse_(page_id, data_dict):
    structured = structure(page_id, data_dict)
    final_dict = construct_dict(merge(structured[2]))
    final_dict["title"] = data_dict[page_id][0]
    final_dict["introduction"] = structured[0]
    final_dict["category"] = structured[1]
    final_dict["id"] = page_id
    #parsed = json.loads(json.dumps(final_dict, ensure_ascii=False, sort_keys=True))
    #print(json.dumps(parsed, indent=4, sort_keys=True))
    return final_dict

In [19]:

inputbuffer = ''
with open(data_path,'r', encoding='utf8') as inputfile:
    append = False
    for line in inputfile:
        if '<page>' in line:
            inputbuffer += line
            append = True
        elif '</page>' in line:
            inputbuffer += line
            append = False

            if len(batches) <= batch_size:
                batches.append(inputbuffer)

            else:
                batch_count += 1
                #print(batches)
                process_batch(batches)
                batches = []

            #inputbuffer = None
            del inputbuffer #probably redundant...
            inputbuffer = ''

        elif append:
            inputbuffer += line


Batch Number: 9 Processed
Indexing Batch Number: 9...
Batch Number: 10 Processed
Indexing Batch Number: 10...
Batch Number: 11 Processed
Indexing Batch Number: 11...
Batch Number: 12 Processed
Indexing Batch Number: 12...
Batch Number: 13 Processed
Indexing Batch Number: 13...
Batch Number: 14 Processed
Indexing Batch Number: 14...
Batch Number: 15 Processed
Indexing Batch Number: 15...
Batch Number: 16 Processed
Indexing Batch Number: 16...
Batch Number: 17 Processed
Indexing Batch Number: 17...
Batch Number: 18 Processed
Indexing Batch Number: 18...
Batch Number: 19 Processed
Indexing Batch Number: 19...
Batch Number: 20 Processed
Indexing Batch Number: 20...
Batch Number: 21 Processed
Indexing Batch Number: 21...
Batch Number: 22 Processed
Indexing Batch Number: 22...
Batch Number: 23 Processed
Indexing Batch Number: 23...
Batch Number: 24 Processed
Indexing Batch Number: 24...
Batch Number: 25 Processed
Indexing Batch Number: 25...
Batch Number: 26 Processed
Indexing Batch Number: 

Batch Number: 154 Processed
Indexing Batch Number: 154...
Batch Number: 155 Processed
Indexing Batch Number: 155...
Batch Number: 156 Processed
Indexing Batch Number: 156...
Batch Number: 157 Processed
Indexing Batch Number: 157...
Batch Number: 158 Processed
Indexing Batch Number: 158...
Batch Number: 159 Processed
Indexing Batch Number: 159...
Batch Number: 160 Processed
Indexing Batch Number: 160...
Batch Number: 161 Processed
Indexing Batch Number: 161...
Batch Number: 162 Processed
Indexing Batch Number: 162...
Batch Number: 163 Processed
Indexing Batch Number: 163...
Batch Number: 164 Processed
Indexing Batch Number: 164...
Batch Number: 165 Processed
Indexing Batch Number: 165...
Batch Number: 166 Processed
Indexing Batch Number: 166...
Batch Number: 167 Processed
Indexing Batch Number: 167...
Batch Number: 168 Processed
Indexing Batch Number: 168...
Batch Number: 169 Processed
Indexing Batch Number: 169...
Batch Number: 170 Processed
Indexing Batch Number: 170...
Batch Number: 

Batch Number: 298 Processed
Indexing Batch Number: 298...
Batch Number: 299 Processed
Indexing Batch Number: 299...
Batch Number: 300 Processed
Indexing Batch Number: 300...
Batch Number: 301 Processed
Indexing Batch Number: 301...
Batch Number: 302 Processed
Indexing Batch Number: 302...
Batch Number: 303 Processed
Indexing Batch Number: 303...
Batch Number: 304 Processed
Indexing Batch Number: 304...
Batch Number: 305 Processed
Indexing Batch Number: 305...
Batch Number: 306 Processed
Indexing Batch Number: 306...
Batch Number: 307 Processed
Indexing Batch Number: 307...
Batch Number: 308 Processed
Indexing Batch Number: 308...
Batch Number: 309 Processed
Indexing Batch Number: 309...
Batch Number: 310 Processed
Indexing Batch Number: 310...
Batch Number: 311 Processed
Indexing Batch Number: 311...
Batch Number: 312 Processed
Indexing Batch Number: 312...
Batch Number: 313 Processed
Indexing Batch Number: 313...
Batch Number: 314 Processed
Indexing Batch Number: 314...
Batch Number: 

Batch Number: 442 Processed
Indexing Batch Number: 442...
Batch Number: 443 Processed
Indexing Batch Number: 443...
Batch Number: 444 Processed
Indexing Batch Number: 444...
Batch Number: 445 Processed
Indexing Batch Number: 445...
Batch Number: 446 Processed
Indexing Batch Number: 446...
Batch Number: 447 Processed
Indexing Batch Number: 447...
Batch Number: 448 Processed
Indexing Batch Number: 448...
Batch Number: 449 Processed
Indexing Batch Number: 449...
Batch Number: 450 Processed
Indexing Batch Number: 450...
Batch Number: 451 Processed
Indexing Batch Number: 451...
Batch Number: 452 Processed
Indexing Batch Number: 452...
Batch Number: 453 Processed
Indexing Batch Number: 453...
Batch Number: 454 Processed
Indexing Batch Number: 454...
Batch Number: 455 Processed
Indexing Batch Number: 455...
Batch Number: 456 Processed
Indexing Batch Number: 456...
Batch Number: 457 Processed
Indexing Batch Number: 457...
Batch Number: 458 Processed
Indexing Batch Number: 458...
Batch Number: 

Batch Number: 585 Processed
Indexing Batch Number: 585...
Batch Number: 586 Processed
Indexing Batch Number: 586...
Batch Number: 587 Processed
Indexing Batch Number: 587...
Batch Number: 588 Processed
Indexing Batch Number: 588...
Batch Number: 589 Processed
Indexing Batch Number: 589...
Batch Number: 590 Processed
Indexing Batch Number: 590...
Batch Number: 591 Processed
Indexing Batch Number: 591...
Batch Number: 592 Processed
Indexing Batch Number: 592...
Batch Number: 593 Processed
Indexing Batch Number: 593...
Batch Number: 594 Processed
Indexing Batch Number: 594...
Batch Number: 595 Processed
Indexing Batch Number: 595...
Batch Number: 596 Processed
Indexing Batch Number: 596...
Batch Number: 597 Processed
Indexing Batch Number: 597...
Batch Number: 598 Processed
Indexing Batch Number: 598...
Batch Number: 599 Processed
Indexing Batch Number: 599...
Batch Number: 600 Processed
Indexing Batch Number: 600...
Batch Number: 601 Processed
Indexing Batch Number: 601...
Batch Number: 

...

Batch Number: 2402 Processed
Indexing Batch Number: 2402...
Batch Number: 2403 Processed
Indexing Batch Number: 2403...
Batch Number: 2404 Processed
Indexing Batch Number: 2404...
Batch Number: 2405 Processed
Indexing Batch Number: 2405...
Batch Number: 2406 Processed
Indexing Batch Number: 2406...
Batch Number: 2407 Processed
Indexing Batch Number: 2407...
Batch Number: 2408 Processed
Indexing Batch Number: 2408...
Batch Number: 2409 Processed
Indexing Batch Number: 2409...
Batch Number: 2410 Processed
Indexing Batch Number: 2410...
Batch Number: 2411 Processed
Indexing Batch Number: 2411...
Batch Number: 2412 Processed
Indexing Batch Number: 2412...
Batch Number: 2413 Processed
Indexing Batch Number: 2413...
Batch Number: 2414 Processed
Indexing Batch Number: 2414...
Batch Number: 2415 Processed
Indexing Batch Number: 2415...
Batch Number: 2416 Processed
Indexing Batch Number: 2416...
Batch Number: 2417 Processed
Indexing Batch Number: 2417...
Batch Number: 2418 Processed
Indexing Ba

Batch Number: 2542 Processed
Indexing Batch Number: 2542...
Batch Number: 2543 Processed
Indexing Batch Number: 2543...
Batch Number: 2544 Processed
Indexing Batch Number: 2544...
Batch Number: 2545 Processed
Indexing Batch Number: 2545...
Batch Number: 2546 Processed
Indexing Batch Number: 2546...
Batch Number: 2547 Processed
Indexing Batch Number: 2547...
Batch Number: 2548 Processed
Indexing Batch Number: 2548...
Batch Number: 2549 Processed
Indexing Batch Number: 2549...
Batch Number: 2550 Processed
Indexing Batch Number: 2550...
Batch Number: 2551 Processed
Indexing Batch Number: 2551...
Batch Number: 2552 Processed
Indexing Batch Number: 2552...
Batch Number: 2553 Processed
Indexing Batch Number: 2553...
Batch Number: 2554 Processed
Indexing Batch Number: 2554...
Batch Number: 2555 Processed
Indexing Batch Number: 2555...
Batch Number: 2556 Processed
Indexing Batch Number: 2556...
Batch Number: 2557 Processed
Indexing Batch Number: 2557...
Batch Number: 2558 Processed
Indexing Ba

Batch Number: 2679 Processed
Indexing Batch Number: 2679...
Batch Number: 2680 Processed
Indexing Batch Number: 2680...
Batch Number: 2681 Processed
Indexing Batch Number: 2681...
Batch Number: 2682 Processed
Indexing Batch Number: 2682...
Batch Number: 2683 Processed
Indexing Batch Number: 2683...
Batch Number: 2684 Processed
Indexing Batch Number: 2684...
Batch Number: 2685 Processed
Indexing Batch Number: 2685...
Batch Number: 2686 Processed
Indexing Batch Number: 2686...
Batch Number: 2687 Processed
Indexing Batch Number: 2687...
Batch Number: 2688 Processed
Indexing Batch Number: 2688...
Batch Number: 2689 Processed
Indexing Batch Number: 2689...
Batch Number: 2690 Processed
Indexing Batch Number: 2690...
Batch Number: 2691 Processed
Indexing Batch Number: 2691...
Batch Number: 2692 Processed
Indexing Batch Number: 2692...
Batch Number: 2693 Processed
Indexing Batch Number: 2693...
Batch Number: 2694 Processed
Indexing Batch Number: 2694...
Batch Number: 2695 Processed
Indexing Ba

Batch Number: 2818 Processed
Indexing Batch Number: 2818...
Batch Number: 2819 Processed
Indexing Batch Number: 2819...
Batch Number: 2820 Processed
Indexing Batch Number: 2820...
Batch Number: 2821 Processed
Indexing Batch Number: 2821...
Batch Number: 2822 Processed
Indexing Batch Number: 2822...
Batch Number: 2823 Processed
Indexing Batch Number: 2823...
Batch Number: 2824 Processed
Indexing Batch Number: 2824...
Batch Number: 2825 Processed
Indexing Batch Number: 2825...
Batch Number: 2826 Processed
Indexing Batch Number: 2826...
Batch Number: 2827 Processed
Indexing Batch Number: 2827...
Batch Number: 2828 Processed
Indexing Batch Number: 2828...
Batch Number: 2829 Processed
Indexing Batch Number: 2829...
Batch Number: 2830 Processed
Indexing Batch Number: 2830...
Batch Number: 2831 Processed
Indexing Batch Number: 2831...
Batch Number: 2832 Processed
Indexing Batch Number: 2832...
Batch Number: 2833 Processed
Indexing Batch Number: 2833...
Batch Number: 2834 Processed
Indexing Ba

Batch Number: 2955 Processed
Indexing Batch Number: 2955...
Batch Number: 2956 Processed
Indexing Batch Number: 2956...
Batch Number: 2957 Processed
Indexing Batch Number: 2957...
Batch Number: 2958 Processed
Indexing Batch Number: 2958...
Batch Number: 2959 Processed
Indexing Batch Number: 2959...
Batch Number: 2960 Processed
Indexing Batch Number: 2960...
Batch Number: 2961 Processed
Indexing Batch Number: 2961...
Batch Number: 2962 Processed
Indexing Batch Number: 2962...
Batch Number: 2963 Processed
Indexing Batch Number: 2963...
Batch Number: 2964 Processed
Indexing Batch Number: 2964...
Batch Number: 2965 Processed
Indexing Batch Number: 2965...
Batch Number: 2966 Processed
Indexing Batch Number: 2966...
Batch Number: 2967 Processed
Indexing Batch Number: 2967...
Batch Number: 2968 Processed
Indexing Batch Number: 2968...
Batch Number: 2969 Processed
Indexing Batch Number: 2969...
Batch Number: 2970 Processed
Indexing Batch Number: 2970...
Batch Number: 2971 Processed
Indexing Ba

In [20]:
keys = [key for key in data_dict]
print(keys)
print(data_dict)
posts = []
process_count = 0
for key in keys:
	#print(data_dict[key])
	process_count += 1
	#print(process_count)
	posts.append(parse_(key, data_dict))

[12588, 392183, 2376935, 2893335, 779238, 1849083, 1278595, 1874673, 5505899, 1821690, 1321902, 1821704, 1322721, 1397666, 2622318, 3923476, 1608529, 896394, 2780551, 1625395, 1609060, 1596901, 3769098, 27604, 29055, 1957865, 396197, 2926062, 2550838, 4439797, 462790, 4699698, 4337576, 4945877, 1506452, 714341, 4489248, 2936431, 2583360, 2926029]
{12588: ['Yodel', 'When you hear the Country and Western singers yodel, do you wish you could do the same thing? Have you tried it, but it never sounded like a real, genuine yodel? The following steps might just help you to be the best yodeler on your block.\n[[Category:Singing]]\n\n== Steps ==\n=== Yodeling Basics ===\n#Understand the break. The secret to yodeling is learning to find the break in your voice. All human voices have two distinct vocal registers; your normal or "chest" voice and your high or "head" voice. The break occurs at the point where your chest voice switches to your head voice. Yodeling is characterized by the jump from c

In [21]:
print(keys)

[12588, 392183, 2376935, 2893335, 779238, 1849083, 1278595, 1874673, 5505899, 1821690, 1321902, 1821704, 1322721, 1397666, 2622318, 3923476, 1608529, 896394, 2780551, 1625395, 1609060, 1596901, 3769098, 27604, 29055, 1957865, 396197, 2926062, 2550838, 4439797, 462790, 4699698, 4337576, 4945877, 1506452, 714341, 4489248, 2936431, 2583360, 2926029]


In [23]:
posts[0]

'Singing'